**STOCK PRICE PREDICTION USING LSTM**

Let's start by installing required packages and import them and the relevant data for stock prices

In [267]:
!pip install yfinance
!pip instal numpy
!pip install tensorflow
!pip install matplotlib
!pip install datetime

import numpy as np
import yfinance as yf
import tensorflow as tf
import os
from matplotlib import pyplot as plt
from datetime import datetime
import pandas as pd
from tensorflow.keras.layers import LSTM,Dense,SimpleRNN,Dropout,Add,Concatenate,Flatten
from tensorflow.keras import Model,Input


You can fill the name of the option as you wish in the next line of code

In this step, TL/Dolar index and BIST 100 index will be used in an autoregressive manner similar to open,high values

In [197]:
cutoff = pd.Timestamp(pd.to_datetime('2020-07-25',format='%Y-%m-%d'),tz='Europe/Istanbul')
info_stock = yf.Ticker("")
data_stock = info_stock.history(period="max")
data_stock = data_stock.sort_values('Date',ascending=True)
info_stock_dolar = yf.Ticker("USDTRY=X")
data_stock_dolar = info_stock_dolar.history(period="max")
data_stock_dolar = data_stock_dolar.drop(['Volume','Dividends','Stock Splits'],axis=1).sort_values('Date',ascending=True)
data_stock_dolar.columns = data_stock_dolar.columns + "_Dolar"
info_stock_bist = yf.Ticker("XU100.IS")
data_stock_bist = info_stock_bist.history(period="max")
data_stock_bist = data_stock_bist.drop(['Volume','Dividends','Stock Splits'],axis=1).sort_values('Date',ascending=True)
data_stock_bist[data_stock_bist.index <= cutoff] = data_stock_bist[data_stock_bist.index <= cutoff]/100
data_stock_bist.columns = data_stock_bist.columns + "_Bist"
data_stock.index = data_stock.index.strftime('%Y-%m-%d %H-%M-%S')
data_stock_dolar.index = data_stock_dolar.index.strftime('%Y-%m-%d %H-%M-%S')
data_stock_bist.index = data_stock_bist.index.strftime('%Y-%m-%d %H-%M-%S')
data_stock = data_stock.merge(data_stock_dolar,how='left',on='Date')
data_stock = data_stock.merge(data_stock_bist,how='left',on='Date')
na_rows = np.invert(np.isnan(data_stock['Open_Dolar']).values)
data_stock = data_stock.iloc[na_rows,:]
data_stock = data_stock.loc[:,['Open','High','Low','Close','Open_Dolar','High_Dolar','Low_Dolar','Close_Dolar','Open_Bist','High_Bist','Low_Bist','Close_Bist','Volume','Dividends','Stock Splits']]
data_stock.index = pd.to_datetime(data_stock.index,format='%Y-%m-%d %H-%M-%S')
data_stock = data_stock.iloc[np.invert(np.isnan(data_stock['Open_Bist']).tolist()),:]

In [268]:
print(type(data_stock))
data_stock.head()

Stock price data is apparent since 2000 on a daily basis. Obviously, average prices of stocks are not provided but open and end prices and lowest and highest prices during the day are available. In addition, days when the dividend payments are made are provided as well as stock splits showing the changes in the issued number of stocks

In [269]:
print(f"Total Number of Dividend Payments: {sum(data_stock['Dividends'].values > 0)}")
print(f"Total Number of Stock Splits: {sum(data_stock['Stock Splits'].values > 0)}")

The changes in the stock prices in Dividend Days or Stock Split Days might be expected to be drastic. Let's see that

In [270]:
dividend_days = [index for index,value in enumerate(data_stock['Dividends'].values) if value > 0]
dividend_selected_days = sorted(np.concatenate([np.array(dividend_days) - 1,np.array(dividend_days),np.array(dividend_days) + 1],axis=0))
data_stock.iloc[np.array(dividend_selected_days),:]

On these two days, it seems that dividend payments were made. There didn't seem to occur any drops/jumps in the prices

In [271]:
split_days = [index for index,value in enumerate(data_stock['Stock Splits'].values) if value > 0]
split_selected_days = sorted(np.concatenate([np.array(split_days[-2:]) - 1,np.array(split_days[-2:]),np.array(split_days[-2:]) + 1],axis=0))
data_stock.iloc[np.array(split_selected_days),:]

On these 2 days, it seems like there occured a stock split which lead to a drastic stock price decrease on 23rd but didn't seem to change the prices on the other split.



In [272]:
def plot_function(seriesx,seriesy,labelx,labely,title):

  for i in range(len(seriesx)):
    if len(seriesx) != 1:
      plt.subplot(1,2,i+1)
    plt.plot(seriesx[i],seriesy[i])
    plt.xlabel(labelx)
    plt.ylabel(labely)
    plt.title(title)
    plt.xticks(rotation=50)

  plt.show()

plt.figure(figsize=(14,5))
plot_function([data_stock.index,data_stock.index[-500:]],[data_stock.Close,data_stock.Close[-500:]],"Date","Stock Price","Date vs. Stock Price")
plt.show()


Let's convert the model to a numpy array and then to tensorflow dataset to transorm the data into window format.

Let's set the batch size to a small number as 2 which is pretty much inline with SGD. In the last experiment, the batch size was quite high making the iterations more stable and faster but the model seemed to get stuck at a local minimum so lowering the batch size might help minimizing loss.

The values should be standardized as min max scaling the values might diminish training pace.

Instead of passing open, close, high etc. as they are, the current value will be passed as the ratio of current value to the previous value i.e., $\frac{y^t_{open}}{y^{t-1}_{open}}$ will be passed in the estimation of $y^t_{close}$ instead of passing $y^t_{open}$ solely.

In [253]:
window_size = 20
batch_size = 100
lastndays = 200
epochs = 100
val_start_date = pd.Timestamp('2023-01-01')
norm_mean_vector = tf.convert_to_tensor(np.mean(data_stock,axis=0).astype('float32'))
norm_std_vector = tf.convert_to_tensor(np.std(data_stock,axis=0).astype('float32'))
close_mean = norm_mean_vector[3]
close_std = norm_std_vector[3]

In [256]:
val_start_date_ix = [x for x,y in enumerate(list(data_stock.index)) if y >= val_start_date][0]
open_series = data_stock['Open'].values
close_series = data_stock['Close'].values
max_close_val = max(close_series)

def convert_data(data_matrix):

  coln = data_matrix.shape[-1]
  rown = data_matrix.shape[-2]
  close_val = data_matrix[-1,3]
  open_val = data_matrix[-1,0]
  data_matrix = tf.concat([data_matrix[1:window_size+1,:12] / data_matrix[:window_size,:12],data_matrix[1:window_size+1,12:]],axis=-1)
  tfz = tf.zeros(shape=[(coln-1)],dtype='float32')
  tfz = tf.concat([data_matrix[-1,0][tf.newaxis],tfz],0)
  tfz2 = tf.zeros(shape=[(window_size-1)],dtype='float32')
  tfz2 = tf.concat([open_val[tf.newaxis],tfz2],0)
  data_matrix = (tf.concat([data_matrix[:-1,:],tfz[tf.newaxis]],0)-norm_mean_vector[None,:])/norm_std_vector[None,:]
  data_matrix = tf.concat([data_matrix,tfz2[:,tf.newaxis]],axis=-1)

  return (data_matrix,close_val)

def preprocess_data(sequence_data, data_type):

  row_sequence = sequence_data.shape[0]
  column_sequence = sequence_data.shape[1]
  sequence_data = np.array(sequence_data,dtype='float32')
  sequence_data = tf.data.Dataset.from_tensor_slices(sequence_data)
  sequence_data = sequence_data.window(size = window_size + 1, shift = 1, drop_remainder = True)
  sequence_data = sequence_data.flat_map(lambda x: x.batch(window_size+1))
  sequence_data = sequence_data.map(lambda x: convert_data(x))
  if data_type == 'training':
    sequence_data = sequence_data.shuffle(row_sequence)
  sequence_data = sequence_data.batch(batch_size).prefetch(1)
  return sequence_data

training_data = preprocess_data(data_stock[:val_start_date_ix],'training')
validation_data = preprocess_data(data_stock[(val_start_date_ix-window_size):],'validation')

for i in validation_data:
  print(f"Shape of first batch of data: {i[0].shape}")
  print(f"Shape of first batch of response data: {i[1].shape}")
  break


Shape of first batch of data: (100, 20, 16)
Shape of first batch of response data: (100,)


As shown above, each batch is composed of 2 observations and 11 days of window with all the previous information to be used. The close value of the day will be estimated using previous 10 days of information in addition to current day's open price as the rest of the information such as close, high, low are masked.

Let's see if using previous close data point in the estimation of next gives reasonable values or not and also let's use current day's open value.

In [273]:
mape_adhoc = tf.keras.metrics.mean_absolute_percentage_error(close_series[1:val_start_date_ix],close_series[:val_start_date_ix-1]).numpy()
print(f"Mean Absolute Percentage All Data Error for Adhoc Method using Close: {mape_adhoc}")
mape_adhoc = tf.keras.metrics.mean_absolute_percentage_error(close_series[val_start_date_ix:],close_series[val_start_date_ix-1:-1]).numpy()
print(f"Mean Absolute Percentage Validation Error for Adhoc Method using Close: {mape_adhoc}")
mape_adhoc = tf.keras.metrics.mean_absolute_percentage_error(close_series[1:val_start_date_ix],open_series[1:val_start_date_ix]).numpy()
print(f"Mean Absolute Percentage All Data Error for Adhoc Method using Open: {mape_adhoc}")
mape_adhoc = tf.keras.metrics.mean_absolute_percentage_error(close_series[val_start_date_ix:],open_series[val_start_date_ix:]).numpy()
print(f"Mean Absolute Percentage Validation Error for Adhoc Method using Open: {mape_adhoc}")

In both training and the validation set, the close values are less accurate in the estimation of next day's close values compared to current day's open values.

Optimizing MAPE should make more sense than any other prevalent metrics such as MSE or MAE as it mimics the return which could be obtained out of investing the derivative. We will adopt a more conservative model with a shallow depth to make the training easier as the gradient will flow through more freely when the network is shallower.



$$
y_{close}^t = y_{open}^{t}\big(1+ r(x^{t-1},x^{t-2},\ldots,x^{t-20})\big)
$$

The formula above shows how the model architecture is designed the $close$ value of the price at time $t$ will be estimated using $open$ price at time $t$ multiplied by $1 + \text{Estimated change in the value of price since the open price}$. $r$ will be estimated using a $tanh$ activation at the top layer assuming the price might go all the way down to 0 and might be doubled where both boundaries are assumed to be extreme.

`ReduceLROnPlateau` will be used in optimizing the loss function as it reduces the learning rate by multiplying the weight by a factor of $f$ which is executed at every $p$ iterations where the loss cannot be minimized further

In [262]:

tf.keras.backend.clear_session()

def model_creation():

  inputs = Input(shape=[window_size,16])
  close_val = tf.expand_dims(inputs[:,0,15],-1)
  inputs2 = inputs[:,:,:-1]
  x = LSTM(128,input_shape=[window_size,15],kernel_initializer = tf.keras.initializers.RandomNormal(mean=0,stddev=1e-3))(inputs2)
  x = Flatten()(x)
  x = Dense(32,activation='relu',kernel_initializer = tf.keras.initializers.RandomNormal(mean=0,stddev=1e-1))(x)
  x = Dropout(0.4)(x)
  x = Dense(32,activation='relu',kernel_initializer = tf.keras.initializers.RandomNormal(mean=0,stddev=1e-1))(x)
  x = Dropout(0.2)(x)
  x = Dense(32,activation='relu',kernel_initializer = tf.keras.initializers.RandomNormal(mean=0,stddev=1e-1))(x)
  x = Dense(1,activation='tanh',kernel_initializer = tf.keras.initializers.RandomNormal(mean=0,stddev=1e-1))(x)

  y = close_val
  x  = y * (1 + x)

  model = Model(inputs=inputs,outputs=x)
  return model

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)

model = model_creation()
plateau_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.9,patience=5,min_lr=1e-10)
model.compile(loss="mape",
              optimizer=optimizer,
              metrics=['mse','mae'])
history = model.fit(training_data,epochs=500,validation_data=validation_data,callbacks=[plateau_lr])

model.save(os.path.join(os.getcwd(),'stock_model.keras'))

Epoch 1/500
45/45 [==============================] - 7s 57ms/step - loss: 1.7097 - mse: 0.1534 - mae: 0.0643 - val_loss: 2.4759 - val_mse: 2.4082 - val_mae: 1.2167 - lr: 0.0100
Epoch 2/500
45/45 [==============================] - 4s 68ms/step - loss: 1.4707 - mse: 0.1457 - mae: 0.0602 - val_loss: 2.3848 - val_mse: 2.3407 - val_mae: 1.1740 - lr: 0.0100
Epoch 3/500
45/45 [==============================] - 3s 44ms/step - loss: 1.4527 - mse: 0.1455 - mae: 0.0597 - val_loss: 2.3662 - val_mse: 2.3299 - val_mae: 1.1656 - lr: 0.0100
Epoch 4/500
45/45 [==============================] - 3s 44ms/step - loss: 1.4666 - mse: 0.1445 - mae: 0.0596 - val_loss: 2.3676 - val_mse: 2.3301 - val_mae: 1.1662 - lr: 0.0100
Epoch 5/500
45/45 [==============================] - 3s 52ms/step - loss: 1.4270 - mse: 0.1435 - mae: 0.0597 - val_loss: 2.4057 - val_mse: 2.3545 - val_mae: 1.1837 - lr: 0.0100
Epoch 6/500
45/45 [==============================] - 3s 45ms/step - loss: 1.4350 - mse: 0.1442 - mae: 0.0598 - val_

In [266]:
training_data_unshuffled = preprocess_data(data_stock[:val_start_date_ix],'validation')
all_predictions = model.predict(training_data_unshuffled).squeeze()
mape_adhoc = tf.keras.metrics.mean_absolute_percentage_error(np.array(close_series[window_size:val_start_date_ix]).squeeze(),all_predictions)
print(f"Mean Absolute Percentage Training Error for LSTM Model: {mape_adhoc}")
validation_data_unshuffled = preprocess_data(data_stock.iloc[(val_start_date_ix-window_size):,:],'validation')
validation_predictions = model.predict(validation_data_unshuffled).squeeze()
mape_adhoc = tf.keras.metrics.mean_absolute_percentage_error(np.array(close_series[val_start_date_ix:]).squeeze(),validation_predictions)
print(f"Mean Absolute Percentage Validation Error for LSTM Model: {mape_adhoc}")

45/45 [==============================] - 3s 55ms/step
Mean Absolute Percentage Training Error for LSTM Model: 1.4016766548156738
3/3 [==============================] - 1s 74ms/step
Mean Absolute Percentage Validation Error for LSTM Model: 2.397963523864746


The training and validation losses are practically the same meaning the information passed as the input is still not sufficient to explaing the response. The next step should be to investigate other financial assets using the same model structure.